# Creation of SQL Database and Tables

## Intro

This notebook documents the design and build of a relational database modeling the activties of a group of hospitals. The aim is to create a structured dataset that captures the key entities and interactions found in a healthcare environment which includes hospitals, doctors, patients, appointments, prescriptions, diseases, and laboratory investigations.

The database is implemented using MySQL, a relational database engine that integrates cleanly with Python. All tables, relationships, and data-generation steps are documented.


## Aims

1. Design a relational schema for modeling the activties of a group of hospitals
2. Implement the tables using SQL CREATE TABLE statements in Python

Later, generate synthetic data to populate the database, and demonstrate key database operations such as inserts and queries 

## Scope

The database includes tables representing:

- Hospitals 
- Doctors – clinical staff working at a hospital
- Patients – individuals receiving care, each assigned to one doctor
- Appointments – events where a doctor sees a patient at a hospital
- Medications – a catalogue of drug names
- Prescriptions – many-to-many link between patients and medications, with doctor + date information
- Diseases – conditions treated by doctors and medications
- DiseaseSpecialist – which doctors specialise in which diseases
- DiseaseTreatment – which medications treat which diseases
- LabTest – a catalogue of test types 
- LabResult – individual patient test results ordered by a doctor

## Database Initialisation

This section sets up the MySQL database that will be used throughout the notebook.
Unlike SQLite, which is file-based, MySQL is a client–server relational database, so the workflow involves connecting to a running MySQL server, creating the database if it does not already exist, and then selecting it for use.

The following initialisation steps are performed:

Import the required Python library (mysql.connector)
This provides the Python interface for connecting to, creating, and querying a MySQL database.

Establish a connection to the MySQL server
Using a username, password, host, and (optionally) database name.
If the database does not yet exist, the notebook will create it.

Create the project database (hospitaldb) if necessary
This ensures the notebook is fully reproducible and can be run on any machine with MySQL installed.

Select the database and create a reusable cursor object
The cursor will be used for executing SQL commands throughout the notebook, including table creation, data insertion, and queries.

Once this initialisation step is complete, the conn (connection) and cur (cursor) objects will be available for all SQL operations in later sections of the notebook. This provides a consistent starting point for defining the schema, populating tables with synthetic data, and running analytical queries.

In [134]:
"""
Creates the MySQL database (`hospitaldb`) if it does not already exist.

This initialisation step connects to the MySQL server using the supplied
credentials and executes a `CREATE DATABASE` statement. Because MySQL
operates as a client–server system rather than a file-based engine,
the database must be explicitly created before any tables can be defined.

Steps performed:
1. Connect to the MySQL server (without selecting a database).
2. Issue a `CREATE DATABASE IF NOT EXISTS` command to ensure the
   `hospitaldb` database is available for use.
3. Close the cursor and connection, as this step only handles
   database creation.

A separate connection—targeting the `hospitaldb` database—will be created
in the next step for running table creation statements and all subsequent
SQL operations.
"""


import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

cur = conn.cursor()
cur.execute("CREATE DATABASE IF NOT EXISTS hospitaldb;")
cur.close()
conn.close()


In [135]:
"""
Initialises the MySQL database connection for the Hospital Information System.

This cell:
- Imports the required Python modules for MySQL connectivity
- Connects to a running MySQL server using user credentials
- Selects (or creates) the target database
- Creates a reusable cursor object for executing SQL statements
- Prints confirmation messages including the MySQL server version

In contrast to SQLite:
- MySQL uses a client–server architecture rather than a file-based database
- Foreign key constraints are always enforced when using the InnoDB engine
- No PRAGMA commands are needed
- Data types and AUTO_INCREMENT behaviour follow MySQL conventions

All subsequent CREATE TABLE and data insertion cells will rely on the
`conn` (connection) and `cur` (cursor) objects defined here.
"""

import mysql.connector
import pandas as pd

# --- Database connection settings (modify as needed) ---
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "hospitaldb"
}

# Establish a connection to the MySQL server and database.
conn = mysql.connector.connect(**db_config)

# Create a cursor object to execute SQL commands.
cur = conn.cursor()

# Show connection confirmation and server version.
cur.execute("SELECT VERSION();")
mysql_version = cur.fetchone()[0]

print("Connected to MySQL database:", db_config["database"])
print("MySQL server version:", mysql_version)



Connected to MySQL database: hospitaldb
MySQL server version: 9.5.0


## HOSPITAL table

This is the core table. Each hospital should have a unique name and a unique id.
There must be attributes address, size and accreditation status.

| Field Name             | Data Type       | Constraints & Notes                            |
| ---------------------- | --------------- | ---------------------------------------------- |
| `hospital_id`          | INT             | **Primary key**, auto-increment                |
| `name`                 | VARCHAR(150)    | **Unique**, not null                           |
| `address`              | VARCHAR(255)    | Not null                                       |
| `size_beds`            | INT             | Not null, must be >0                           |
| `accreditation_status` | VARCHAR(50)     | e.g. "Accredited", "Pending", "Not accredited" |


In [136]:
"""
Creates the `Hospital` table in the MySQL database.

This table stores core information about each hospital, including:
- A unique primary key (`hospital_id`)
- A unique hospital name
- Address information
- Bed capacity (`size_beds`)
- Type of hospital (e.g., General, Teaching, Specialist)
- Accreditation status

This version is written for MySQL, which requires:
- Explicit VARCHAR length limits
- InnoDB storage engine for future foreign key support

The table is only created if it does not already exist.
"""

sql_create_hospital = """
CREATE TABLE IF NOT EXISTS Hospital (
    hospital_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(150) NOT NULL UNIQUE,
    address VARCHAR(255) NOT NULL,
    size_beds INT NOT NULL CHECK (size_beds > 0),
    hospital_type VARCHAR(100),
    accreditation_status VARCHAR(100)
) ENGINE=InnoDB;
"""

cur.execute(sql_create_hospital)
conn.commit()
print("Hospital table created.")


Hospital table created.


## DOCTOR table

This table contains a list of doctors. Related information should be stored about each doctor including at least:
name, date of birth, Address.

1 hospital can have a relationship with many doctors and the hospital_id field allows this (foreign key) 

| Field Name      | Data Type       | Constraints / Notes                                    |
| --------------- | --------------- | ------------------------------------------------------ |
| `doctor_id`     | INT             | **Primary key**, auto-increment                        |
| `hospital_id`   | INT             | **Foreign key** = Hospital.hospital_id                 |
| `first_name`    | VARCHAR(100)    | Not null                                               |
| `last_name`     | VARCHAR(100)    | Not null                                               |
| `date_of_birth` | DATE            | Not null                                               |
| `specialty`     | VARCHAR(100)    | e.g. Cardiology, Oncology                              |
| `address`       | VARCHAR(255)    |  Not Null                                              |


In [ ]:
"""
Creates the `Doctor` table in the MySQL database.

This table stores demographic and professional information for each doctor.
Each doctor is assigned to exactly one hospital, enforced by the foreign key
constraint on `hospital_id`. This version is adapted for MySQL, which enforces
data types strictly and requires VARCHAR lengths and AUTO_INCREMENT syntax.

Fields:
- doctor_id:      Primary key (AUTO_INCREMENT)
- hospital_id:    Foreign key referencing Hospital(hospital_id)
- first_name:     Doctor's given name (required)
- last_name:      Doctor's surname (required)
- date_of_birth:  Stored as a MySQL DATE type (required)
- specialty:      Medical specialty (optional)
- address:        Work or home address (required)

"""

sql_create_doctor = """
CREATE TABLE IF NOT EXISTS Doctor (
    doctor_id INT AUTO_INCREMENT PRIMARY KEY,
    hospital_id INT NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    date_of_birth DATE NOT NULL,
    specialty VARCHAR(100),
    address VARCHAR(255) NOT NULL,
    FOREIGN KEY (hospital_id)
        REFERENCES Hospital(hospital_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
) ENGINE=InnoDB;
"""

cur.execute(sql_create_doctor)
conn.commit()
print("Doctor table created.")


Doctor table created.


## PATIENT TABLE

Each patient is assigned to one doctor (foreign key)

| Field Name      | Data Type (SQL) | Constraints / Notes                |
| --------------- | --------------- | ---------------------------------- |
| `patient_id`    | INT             | **Primary key**, auto-increment    |
| `doctor_id`     | INT             | **Foreign key** =  Doctor.doctor_id |
| `first_name`    | VARCHAR(100)    | Not null                           |
| `last_name`     | VARCHAR(100)    | Not null                           |
| `date_of_birth` | DATE            | Not null                           |
| `address`       | VARCHAR(255)    | Not null                           |       
| `gender`        | VARCHAR(10)     | Not null                           |


In [ ]:
"""
Creates the `Patient` table in the MySQL database.

This table stores demographic information for individual patients within the
hospital system. Each patient is assigned to exactly one doctor, represented by
a foreign key reference to `Doctor.doctor_id`. This reflects the requirement
that every patient in the dataset has a single responsible clinician.

Fields:
- patient_id:     Primary key (AUTO_INCREMENT)
- doctor_id:      Foreign key referencing Doctor.doctor_id
- first_name:     Patient’s given name (required)
- last_name:      Patient’s surname (required)
- date_of_birth:  Stored as a MySQL DATE type (required)
- address:        Residential address for correspondence (required)
- gender:         Patient’s gender (stored as a string "Male", "Female", "Other")

"""

sql_create_patient = """
CREATE TABLE IF NOT EXISTS Patient (
    patient_id INT AUTO_INCREMENT PRIMARY KEY,
    doctor_id INT NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,       
    date_of_birth DATE NOT NULL,
    address VARCHAR(255) NOT NULL,
    gender ENUM('Male', 'Female', 'Other') NOT NULL,
    FOREIGN KEY (doctor_id)
        REFERENCES Doctor(doctor_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
) ENGINE=InnoDB;
"""
cur.execute(sql_create_patient)
conn.commit()
print("Patient table created.")

Patient table created.


## MEDICATION table

30+ meds. A medication can be prescribed to multiple patients and each patient could be prescribed multiple medications, but that many-to-many relationship will be described in the PRESCRIPTIONS table.


| Field Name.      | Data Type       | Constraints/ Notes               |
| ---------------- | --------------- | -------------------------------- |
| `medication_id`  | INT             | **primary key** autoincrement.   |
| `name`           | VARCHAR(150).   | Unique, not null.                |

In [ ]:
"""
Creates the `Medication` table in the MySQL database.

This table stores a list of medications that may be prescribed to patients
within the hospital system. Each medication is identified by a unique
auto-incrementing primary key, and each medication name must be unique.
The many-to-many relationship between patients and medications is handled
not in this table, but in the `Prescription` table, where each prescription
links a single patient to a single medication on a specific date.

Fields:
- medication_id:  Primary key (AUTO_INCREMENT).
- name:           Medication name (required, unique), stored as a VARCHAR(150).

Notes:
- Only the core medication attributes are stored here; prescribing events
  are represented in the `Prescription` table.
- Ensuring medication names are unique prevents duplication.
"""

sql_create_medication = """
CREATE TABLE IF NOT EXISTS Medication (
    medication_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(150) NOT NULL UNIQUE
) ENGINE=InnoDB;
"""

cur.execute(sql_create_medication)
conn.commit()
print("Medication table created.")


Medication table created.


## PRESCRIPTTION Table

This is a Junction table for PATIENT and MEDICATION

This table represents the many-to-many relationship between patients and medications. Each record indicates that a specific patient has been prescribed a specific medication.
It also records who prescribed the medication, when it was prescrinbed, and will contain extra information like dose, duration and route. This is called a descriptive junction table, or a bridge entity with attributes.

| Field Name          | Data Type    | Notes                               |
| ------------------- | ------------ | ----------------------------------- |
| `prescription_id`   | INT          | **Primary key**                     |
| `patient_id`        | INT          | **FK = Patient**                    |
| `doctor_id`         | INT          | **FK = Doctor**                     |
| `medication_id`     | INT          | **FK = Medication**                 |
| `prescribed_date`   | DATE         | Must be within past 2 years         |
| `dose_instructions` | VARCHAR(255) | e.g., "Take one tablet twice a day" |
| `duration_days`     | INT          | Optional                            |
| `route`             | VARCHAR(150) | not Null                            |


In [ ]:
"""
Creates the `Prescription` table in the MySQL database.

This table represents a descriptive junction (bridge) entity linking
patients, medications, and doctors. It captures the many-to-many relationship
between patients and medications, with information about
the prescription event.

Each record represents a single prescription written for a specific patient,
for a specific medication, by a specific doctor, on a specific date.
Additional descriptive fields such as dose instructions, duration, and route
allow the table to store prescribing information.

Fields:
- prescription_id:   Primary key (AUTO_INCREMENT).
- patient_id:        Foreign key referencing Patient.patient_id.
- doctor_id:         Foreign key referencing Doctor.doctor_id.
- medication_id:     Foreign key referencing Medication.medication_id.
- prescribed_date:   Date the prescription was issued (required).
- dose_instructions: Free-text dosage instructions (optional but common).
- duration_days:     Intended duration of the course in days (optional).
- route:             Route of administration (e.g., 'Oral', 'IV') (required).

Notes:
- The table itself does not ensure that `prescribed_date`
  falls within the required 2-year window. Application code should enforce this.
"""

sql_create_prescription = """
CREATE TABLE IF NOT EXISTS Prescription (
    prescription_id INT AUTO_INCREMENT PRIMARY KEY,
    patient_id INT NOT NULL,
    doctor_id INT NOT NULL,
    medication_id INT NOT NULL,
    prescribed_date DATE NOT NULL,
    dose_instructions VARCHAR(255),
    duration_days INT,
    route VARCHAR(150) NOT NULL,

    FOREIGN KEY (patient_id)
        REFERENCES Patient(patient_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    FOREIGN KEY (doctor_id)
        REFERENCES Doctor(doctor_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    FOREIGN KEY (medication_id)
        REFERENCES Medication(medication_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
) ENGINE=InnoDB;
"""

cur.execute(sql_create_prescription)
conn.commit()
print("Prescription table created.")


Prescription table created.



## DISEASES Table

One disease can be treated by many meds, and by many doctors. 
One medication can treat many diseases.
One doctor can specialise in many diseases.

There are TWO many-to-many relationships here, which can be represented in two junction tables: Disease-Medication, and Disease-Doctor


| Field         | Type         | Notes                  |
| ------------- | ------------ | ---------------------- |
| `disease_id`  | INT          | **Primary key**        |
| `name`        | VARCHAR(150) | Unique, not null       |
| `description` | TEXT         | Optional               |
| `icd10_code`  | VARCHAR(10)  | Optional               |




In [ ]:
"""
Creates the `Disease` table in the MySQL database.

This table stores a master list of diseases or clinical conditions recognised
within the hospital system. A disease may be treated with multiple medications,
and multiple doctors may specialise in its management. These two many-to-many
relationships are represented separately in the `DiseaseTreatment` and
`DiseaseSpecialist` junction tables, rather than in this core disease table.

Fields:
- disease_id:   Primary key (AUTO_INCREMENT).
- name:         Name of the disease/condition (required, unique).
- description:  Optional free-text description or summary of the condition.
- icd10_code:   Optional ICD-10 diagnostic code for standardised classification.

Notes:
- Only core disease attributes are stored here. Relational mappings to doctors
  and medications are handled in dedicated junction tables.
"""

sql_create_disease = """
CREATE TABLE IF NOT EXISTS Disease (
    disease_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(150) NOT NULL UNIQUE,
    description TEXT,
    icd10_code VARCHAR(10)
) ENGINE=InnoDB;
"""

cur.execute(sql_create_disease)
conn.commit()
print("Disease table created.")


Disease table created.


## DiseaseMEDICATION

Many to many, represents which meds are used to treat which diseases

| Field                  | Type | Notes           |
| ---------------------- | ---- | --------------- |
| `disease_treatment_id` | INT  | Primary Key             |
| `disease_id`           | INT  | Foreign Key - Disease    |
| `medication_id`        | INT  | Foreign Key - Medication |


In [142]:
"""
Creates the `DiseaseTreatment` junction table in the MySQL database.

This table represents the many-to-many relationship between diseases and
medications. A single disease may be treated with multiple medications, and
a single medication may be used to treat multiple diseases. 

Fields:
- disease_treatment_id:  Primary key (AUTO_INCREMENT).
- disease_id:            Foreign key referencing Disease.disease_id.
- medication_id:         Foreign key referencing Medication.medication_id.

Notes:
- The combination of (disease_id, medication_id) should be unique to
  prevent duplicate treatment mappings.
"""

sql_create_disease_treatment = """
CREATE TABLE IF NOT EXISTS DiseaseTreatment (
    disease_treatment_id INT AUTO_INCREMENT PRIMARY KEY,
    disease_id INT NOT NULL,
    medication_id INT NOT NULL,

    FOREIGN KEY (disease_id)
        REFERENCES Disease(disease_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    FOREIGN KEY (medication_id)
        REFERENCES Medication(medication_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    UNIQUE (disease_id, medication_id)
) ENGINE=InnoDB;
"""

cur.execute(sql_create_disease_treatment)
conn.commit()
print("DiseaseTreatment table created.")


DiseaseTreatment table created.


## DiseaseDOCTOR

Many to Many, represents which doctors treat which diseases 

| Field                   | Type | Notes        |
| ----------------------- | ---- | ------------ |
| `disease_specialist_id` | INT  | PK           |
| `disease_id`            | INT  | FK → Disease |
| `doctor_id`             | INT  | FK → Doctor  |



In [143]:
"""
Creates the `DiseaseSpecialist` junction table in the MySQL database.

This table represents the many-to-many relationship between diseases and doctors.
A single disease may be managed by multiple doctors (different specialists), and
each doctor may specialise in multiple diseases. This bridge table captures these
relationships in a fully normalised form.

Fields:
- disease_specialist_id:  Primary key (AUTO_INCREMENT).
- disease_id:             Foreign key referencing Disease.disease_id.
- doctor_id:              Foreign key referencing Doctor.doctor_id.

Notes:
- The combination (disease_id, doctor_id) is enforced as UNIQUE to eliminate
  duplicate specialisation entries.
"""

sql_create_disease_specialist = """
CREATE TABLE IF NOT EXISTS DiseaseSpecialist (
    disease_specialist_id INT AUTO_INCREMENT PRIMARY KEY,
    disease_id INT NOT NULL,
    doctor_id INT NOT NULL,

    FOREIGN KEY (disease_id)
        REFERENCES Disease(disease_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    FOREIGN KEY (doctor_id)
        REFERENCES Doctor(doctor_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    UNIQUE (disease_id, doctor_id)
) ENGINE=InnoDB;
"""

cur.execute(sql_create_disease_specialist)
conn.commit()
print("DiseaseSpecialist table created.")


DiseaseSpecialist table created.


## APPOINTMENTS table

Each Appointment is a one to one relationship between a PATIENT and a DOCTOR occuring at a HOSPTIAL and at a certain time and for a certain duration.

| Field               | Data Type    | Notes                                      |
| ------------------- | ------------ | ------------------------------------------ |
| `appointment_id`    | INT          | **Primary key**                            |
| `patient_id`        | INT          | **FK → Patient**                           |
| `doctor_id`         | INT          | **FK → Doctor**                            |
| `hospital_id`       | INT          | **FK → Hospital**                          |
| `appointment_start` | DATETIME     | exact scheduled time                       |
| `duration_minutes`  | INT          | typical values 10–60                       |
| `reason`            | VARCHAR(255) | optional free-text                         |
| `status`            | VARCHAR(50)  | e.g. "Scheduled", "Completed", "Cancelled" |


In [144]:
"""
Creates the `Appointment` table in the MySQL database.

This table represents scheduled clinical encounters between a patient and a
doctor, occurring at a specific hospital location. Each appointment records
the start time, duration, and optional descriptive fields such as the reason
for the visit and its current status.

Each appointment links:
- exactly one patient,
- to exactly one doctor,
- at exactly one hospital,
- on a specific date and time.

Fields:
- appointment_id:     Primary key (AUTO_INCREMENT).
- patient_id:         Foreign key referencing Patient.patient_id.
- doctor_id:          Foreign key referencing Doctor.doctor_id.
- hospital_id:        Foreign key referencing Hospital.hospital_id.
- appointment_start:  Scheduled start time, stored as DATETIME.
- duration_minutes:   Duration of the appointment in minutes (typical range 10–60).
- reason:             Optional free-text explanation of visit purpose.
- status:             Appointment state (e.g., 'Scheduled', 'Completed', 'Cancelled').

"""

sql_create_appointment = """
CREATE TABLE IF NOT EXISTS Appointment (
    appointment_id INT AUTO_INCREMENT PRIMARY KEY,
    patient_id INT NOT NULL,
    doctor_id INT NOT NULL,
    hospital_id INT NOT NULL,
    appointment_start DATETIME NOT NULL,
    duration_minutes INT NOT NULL,
    reason VARCHAR(255),
    status VARCHAR(50),

    FOREIGN KEY (patient_id)
        REFERENCES Patient(patient_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    FOREIGN KEY (doctor_id)
        REFERENCES Doctor(doctor_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    FOREIGN KEY (hospital_id)
        REFERENCES Hospital(hospital_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
) ENGINE=InnoDB;
"""

cur.execute(sql_create_appointment)
conn.commit()
print("Appointment table created.")



Appointment table created.


## LABTESTS table

A table should be created for storing the results from lab tests,
which are for individual patients.
A patient may have many lab tests, but each lab result is for one patient.
and each lab test should be requested by a specific doctor (one to one)
each lab result corresponds to one test type (many to one)

To achieve this we can create a LAB TESTStable, and then a LABRESULTS table which is a junctional table with attributes, joining a test result to a patient (many to one) and describing the doctor who requested it, the date of the request, the result of the test, the date of the result, and optionally whether it is normal or abnormal and a reference range.

| Field             | Type         | Notes                                              |
| ----------------- | ------------ | -------------------------------------------------- |
| `lab_test_id`     | INT          | **Primary key**                                    |
| `name`            | VARCHAR(150) | e.g. “HbA1c”, “CRP”                                |
| `description`     | TEXT         | optional                                           |
| `units`           | VARCHAR(20)  | e.g. “mmol/mol”, “mg/L”                            |
| `reference_range` | VARCHAR(50)  |                                                    |
| `sample_type`     | VARCHAR(50)  | e.g. “Blood”, “Urine”                              |


In [145]:
"""
Creates the `LabTest` table in the MySQL database.

This table defines the catalogue of laboratory test types available within the
hospital system. Each row represents a type of test (e.g., HbA1c, CRP, FBC),
including its name, description, measurement units, reference range, and the
biological sample required.

This table does *not* store patient-specific results. Laboratory results for
individual patients are stored in the `LabResult` descriptive junction table,
which links a patient, a test type, and the requesting doctor together with
result values and timestamps.

Fields:
- lab_test_id:      Primary key (AUTO_INCREMENT).
- name:             Name of the test (required, unique).
- description:      Optional free-text description of what the test measures.
- units:            Measurement units (e.g., mg/L, mmol/mol).
- reference_range:  Expected normal range for the test (string representation).
- sample_type:      Type of biological sample required (e.g., 'Blood', 'Urine').

Notes:
- This table stores metadata about laboratory tests; actual patient results
  belong in the `LabResult` table.
- Test names are enforced as UNIQUE to prevent duplication.
"""

sql_create_labtest = """
CREATE TABLE IF NOT EXISTS LabTest (
    lab_test_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(150) NOT NULL UNIQUE,
    description TEXT,
    units VARCHAR(20),
    reference_range VARCHAR(50),
    sample_type VARCHAR(50)
) ENGINE=InnoDB;
"""

cur.execute(sql_create_labtest)
conn.commit()
print("LabTest table created.")


LabTest table created.


## LABRESULTS Table

| Field            | Type        | Notes                            |
| ---------------- | ----------- | -------------------------------- |
| `lab_result_id`  | INT         | **Primary key**                  |
| `lab_test_id`    | INT         | FK - LabTest                     |
| `patient_id`     | INT         | FK - Patient                     |
| `doctor_id`      | INT         | FK - Doctor                      |
| `requested_date` | DATE        | when the doctor ordered the test |
| `result_date`    | DATE        | when the lab produced the result |
| `result_value`   | VARCHAR(50) | numeric or text result           |
| `is_normal`      | BOOLEAN     | optional                         |
| `notes`          | TEXT        | optional commentary              |


In [146]:
"""
Creates the `LabResult` table in the MySQL database.

This table stores individual laboratory results for patients. Each record
represents one completed laboratory test, linking together:

- the patient who the test was performed on,
- the doctor who requested the test,
- the type of test performed (from the LabTest catalogue).

The table also records key times (requested date and result date),
the measured result value, whether the result is interpreted as normal/abnormal,
and optional free-text notes.

This table is a descriptive junction table between Patient, Doctor, and LabTest.

Fields:
- lab_result_id:   Primary key (AUTO_INCREMENT).
- lab_test_id:     Foreign key referencing LabTest.lab_test_id.
- patient_id:      Foreign key referencing Patient.patient_id.
- doctor_id:       Foreign key referencing Doctor.doctor_id.
- requested_date:  Date the test was ordered (required).
- result_date:     Date the result became available (optional but typical).
- result_value:    Result as text (supports numeric or textual values).
- is_normal:       Boolean flag indicating normal/abnormal interpretation.
- notes:           Optional free-text commentary.

Notes:
- BOOLEAN in MySQL is implemented as TINYINT(1). Either value is accepted.
- CASCADE rules ensure results are deleted when a parent record is removed.
"""

sql_create_labresult = """
CREATE TABLE IF NOT EXISTS LabResult (
    lab_result_id INT AUTO_INCREMENT PRIMARY KEY,
    lab_test_id INT NOT NULL,
    patient_id INT NOT NULL,
    doctor_id INT NOT NULL,
    requested_date DATE NOT NULL,
    result_date DATE,
    result_value VARCHAR(50),
    is_normal BOOLEAN,
    notes TEXT,

    FOREIGN KEY (lab_test_id)
        REFERENCES LabTest(lab_test_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    FOREIGN KEY (patient_id)
        REFERENCES Patient(patient_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE,

    FOREIGN KEY (doctor_id)
        REFERENCES Doctor(doctor_id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
) ENGINE=InnoDB;
"""

cur.execute(sql_create_labresult)
conn.commit()
print("LabResult table created.")


LabResult table created.


### Inspect the database that has been created

In [ ]:
# Inspect the database

cur.execute("SHOW TABLES;")
for table in cur.fetchall():
    print(table)


('Appointment',)
('Disease',)
('DiseaseSpecialist',)
('DiseaseTreatment',)
('Doctor',)
('Hospital',)
('LabResult',)
('LabTest',)
('Medication',)
('Patient',)
('Prescription',)


In [ ]:
# Inspect the tables and their structures

cur.execute("SHOW TABLES;")
tables = [t[0] for t in cur.fetchall()]

for table in tables:
    print(f"\n=== Columns for {table} ===")
    cur.execute(f"DESCRIBE {table};")
    for column in cur.fetchall():
        print(column)



=== Columns for Appointment ===
('appointment_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('patient_id', 'int', 'NO', 'MUL', None, '')
('doctor_id', 'int', 'NO', 'MUL', None, '')
('hospital_id', 'int', 'NO', 'MUL', None, '')
('appointment_start', 'datetime', 'NO', '', None, '')
('duration_minutes', 'int', 'NO', '', None, '')
('reason', 'varchar(255)', 'YES', '', None, '')
('status', 'varchar(50)', 'YES', '', None, '')

=== Columns for Disease ===
('disease_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('name', 'varchar(150)', 'NO', 'UNI', None, '')
('description', 'text', 'YES', '', None, '')
('icd10_code', 'varchar(10)', 'YES', '', None, '')

=== Columns for DiseaseSpecialist ===
('disease_specialist_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('disease_id', 'int', 'NO', 'MUL', None, '')
('doctor_id', 'int', 'NO', 'MUL', None, '')

=== Columns for DiseaseTreatment ===
('disease_treatment_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('disease_id', 'int', 'NO', 'MUL', 